In [6]:
from premise import *
import bw2data
bw2data.projects.set_current("maria")
#bw2data.projects.set_current("ecoinvent-3.8-cutoff")
#bw2data.projects.set_current("ecoinvent-3.9.1-cutoff_bw25")
#clear_inventory_cache()
scenarios = [
    #{"model": "remind", "pathway": "SSP2-NDC", "year": 2025},
    {"model": "remind", "pathway": "SSP2-NDC", "year": 2035},
    {"model": "remind", "pathway": "SSP2-NDC", "year": 2050},
    {"model": "remind", "pathway": "SSP3-rollBack", "year": 2025},
    {"model": "remind", "pathway": "SSP3-rollBack", "year": 2035},
    {"model": "remind", "pathway": "SSP3-rollBack", "year": 2050},
    #{"model": "remind", "pathway": "SSP2-NPi", "year": 2030},
    #{"model": "remind", "pathway": "SSP2-NPi", "year": 2040},
    #{"model": "remind", "pathway": "SSP2-NPi", "year": 2050},
    #{"model": "remind", "pathway": "SSP2-NPi", "year": 2060},
    #{"model": "remind", "pathway": "SSP2-PkBudg1150", "year": 2020},
    #{"model": "witch", "pathway": "SSP2-Base", "year": 2050},
    #{"model": "witch", "pathway": "SSP2-PkBudg1150", "year": 2050},
    #{"model": "image", "pathway": "SSP1-L", "year": 2030},
    #{"model": "image", "pathway": "SSP1-Ma", "year": 2030},
    #{"model": "image", "pathway": "SSP1-VLLO", "year": 2030},
    #{"model": "image", "pathway": "SSP2-L", "year": 2030},
    #{"model": "image", "pathway": "SSP2-M", "year": 2030},
    #{"model": "image", "pathway": "SSP2-VLHO", "year": 2040},
    #{"model": "image", "pathway": "SSP3-H", "year": 2030},
    #{"model": "image", "pathway": "SSP5-H", "year": 2030},
    #{"model": "remind", "pathway": "SSP1-PkBudg1000", "year": 2040},
    #{"model": "remind", "pathway": "SSP2-PkBudg1000", "year": 2040},
    #{"model": "remind", "pathway": "SSP2-PkBudg650", "year": 2060},
    #{"model": "remind", "pathway": "SSP2-rollBack", "year": 2060},
    #{"model": "remind", "pathway": "SSP2-NDC", "year": 2060},
    #{"model": "remind", "pathway": "SSP3-rollBack", "year": 2060},
    #{"model": "remind-eu", "pathway": "SSP2-PkBudg1000", "year": 2060},
    #{"model": "tiam-ucl", "pathway": "SSP2-RCP45", "year": 2060},
]

ndb = NewDatabase(
        scenarios=scenarios,
        source_db="ecoinvent-3.11-cutoff",
        source_version="3.11",
        biosphere_name="biosphere3",
        key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=',
        keep_imports_uncertainty=True,
        keep_source_db_uncertainty=True
)
#ndb.write_db_to_brightway("test final energy 2")

premise v.(2, 3, 0, 'dev0')
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-----

In [ ]:
ndb.update()

Processing scenarios for all sectors:   0%|     | 0/5 [00:00<?, ?it/s]

In [ ]:
ndb.write_db_to_brightway()

In [3]:
from premise.activity_maps import InventorySet, debug_mapping_to_dataframe
from premise.utils import load_database

for scenario in ndb.scenarios:
    scenario = load_database(scenario)

In [4]:
    df = debug_mapping_to_dataframe(
    scenarios[0]
)

In [6]:
df.to_excel("mapping.xlsx")

In [ ]:
inv = InventorySet(
    database=ndb.scenarios[0]["database"],
    version="3.10",
    model="remind"
)
biomass_map = inv.generate_cement_map()
df_biomass = debug_mapping_to_dataframe(biomass_map)
df_biomass

In [12]:
dataset = bw2data.Database("ecoinvent-3.11-cutoff").random()

list = []

for exc in dataset.exchanges():
    list.append(
        (
            dataset["name"],
            dataset.get("comment", ""),
            exc["name"],
            exc.input.get("reference product"),
            exc.input.get("location"),
            exc["type"],
            exc["amount"],
            exc["unit"],
        )
    )

In [13]:
import pandas as pd
pd.DataFrame(list, columns=["dataset", "comment","name", "product", "location", "type", "amount", "unit"])

,dataset,comment,name,product,location,type,amount,unit
0,"natural gas, high pressure, import from MY",This dataset represents the import of natural ...,"natural gas, high pressure","natural gas, high pressure",KR,production,1.000000e+00,cubic meter
1,"natural gas, high pressure, import from MY",This dataset represents the import of natural ...,"natural gas, burned in gas turbine","natural gas, burned in gas turbine",MY,technosphere,8.100000e-02,megajoule
2,"natural gas, high pressure, import from MY",This dataset represents the import of natural ...,"natural gas, high pressure","natural gas, high pressure",KR,technosphere,1.000100e+00,cubic meter
3,"natural gas, high pressure, import from MY",This dataset represents the import of natural ...,Butane,None,None,biosphere,6.352050e-07,kilogram
4,"natural gas, high pressure, import from MY",This dataset represents the import of natural ...,"Carbon dioxide, fossil",None,None,biosphere,2.292972e-06,kilogram
5,"natural gas, high pressure, import from MY",This dataset represents the import of natural ...,Ethane,None,None,biosphere,5.490360e-06,kilogram
6,"natural gas, high pressure, import from MY",This dataset represents the import of natural ...,Mercury II,None,None,biosphere,1.000000e-12,kilogram
7,"natural gas, high pressure, import from MY",This dataset represents the import of natural ...,"Methane, fossil",None,None,biosphere,6.628665e-05,kilogram
8,"natural gas, high pressure, import from MY",This dataset represents the import of natural ...,"NMVOC, non-methane volatile organic compounds",None,None,biosphere,4.573800e-08,kilogram
9,"natural gas, high pressure, import from MY",This dataset represents the import of natural ...,Propane,None,None,biosphere,1.235250e-06,kilogram


In [10]:
for ds in ndb.scenarios[3]["database"]:
    if ds["name"] == "transport, passenger bus, gasoline, 13m single deck urban bus, EURO-VI":
        print(ds["location"])

RER
AFR
AUS
CAN
CHI
CSA
EEU
FSU
IND
JPN
MEA
MEX
ODA
SKO
UK
USA
WEU
World


In [6]:
t = [
    tech for tech in ndb.scenarios[0]["iam data"].daccs_energy_use.variables.values
]

In [7]:
[x for x in t if x not in ndb.scenarios[0]["iam data"].production_volumes.variables.values]

[]

In [9]:
ndb.scenarios[0]["iam data"].production_volumes.sel(
    year=2060, variables=t
)

<xarray.DataArray 'value' (region: 13, variables: 4)> Size: 416B
array([[0.e+00, 0.e+00, 0.e+00, 0.e+00],
       [0.e+00, 0.e+00, 0.e+00, 0.e+00],
       [0.e+00, 0.e+00, 0.e+00, 0.e+00],
       [0.e+00, 0.e+00, 0.e+00, 0.e+00],
       [0.e+00, 0.e+00, 0.e+00, 0.e+00],
       [0.e+00, 0.e+00, 0.e+00, 0.e+00],
       [0.e+00, 0.e+00, 0.e+00, 0.e+00],
       [0.e+00, 0.e+00, 0.e+00, 0.e+00],
       [0.e+00, 0.e+00, 0.e+00, 0.e+00],
       [0.e+00, 0.e+00, 0.e+00, 0.e+00],
       [0.e+00, 0.e+00, 0.e+00, 0.e+00],
       [0.e+00, 0.e+00, 0.e+00, 0.e+00],
       [0.e+00, 2.e-07, 1.e-07, 0.e+00]])
Coordinates:
  * region     (region) object 104B 'CAZ' 'CHA' 'EUR' ... 'SSA' 'USA' 'World'
    year       int64 8B 2060
  * variables  (variables) <U53 848B 'energy, for DACCS, from hydrogen turbin...
Attributes:
    unit:     {'Carbon Management|Carbon Capture|Industrial Processes|Industr...

In [5]:
 ndb.scenarios[0]["iam data"].daccs_energy_use.sel(region="CAZ", year=2060)

<xarray.DataArray 'value' (variables: 4)> Size: 32B
array([0., 0., 0., 0.])
Coordinates:
    region     <U3 12B 'CAZ'
    year       int64 8B 2060
  * variables  (variables) <U40 640B 'energy, for DACCS, from electricity' .....
Attributes:
    unit:     {'FE|CDR|DAC|+|Electricity': 'EJ/yr', 'FE|CDR|DAC|+|Gases': 'EJ...

In [2]:
    ei311 = list(set([
    (
        act["name"],
        act["reference product"],
    )
    for act in bw2data.Database("ecoinvent-3.11-cutoff")
]))